In [2]:
using StaticArrays
# using Plots
using LinearAlgebra
# using Makie
using GLMakie 
using LsqFit
include("Elastic.jl")
using .Elastic
using FFMPEG

In [3]:
# 定义铜的晶格常数（单位：Å）
lattice_constant = 1.0

# 定义铜的FCC晶胞的基矢量
lattice_vectors = collect((Matrix([
    lattice_constant 0.0 0.0; #a1
    0.0 lattice_constant 0.0; #a2
    0.0 0.0 lattice_constant] #a3
))')

# 定义铜的FCC晶胞中的原子位置（单位：Å）
atom_positions = [
    Vector([0.0, 0.0, 0.0]),
    Vector([0.0, 0.5, 0.5]),
    Vector([0.5, 0.0, 0.5]),
    Vector([0.5, 0.5, 0.0]),
    Vector([1.0, 0.0, 0.0]),
    Vector([0.0, 1.0, 0.0]),
    Vector([0.0, 0.0, 1.0]),
    Vector([0.5, 1.0, 0.5]),
    Vector([1.0, 0.5, 0.5]),
    Vector([0.5, 0.5, 1.0]),
    Vector([1.0, 0.0, 1.0]),
    Vector([1.0, 1.0, 0.0]),
    Vector([0.0, 1.0, 1.0]),
    Vector([1.0, 1.0, 1.0])
] 

# 创建铜的原子列表
atoms = [Atom(pos) for pos in atom_positions]

cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,3,3,3)
fcell=filtercell(cpcell)
fig=visualize_unitcell_atoms(fcell)

In [4]:
#lj势能
function lj(r::Float64)
    return 4*(1/r^12-1/r^6)
end
function Flj(r::Vector{Float64})
    rn=norm(r)
    return 24*(2/rn^14-1/rn^8)*r
end

ct=5.0
interaction = Interaction(lj, Flj, ct, 0.1)

x=1:0.001:ct
y=interaction.cutenergy.(x)
lines(x,y)


In [5]:
visualize_unitcell_atoms0(fcell,iftext=true)
# cell_forcei!(fcell,interaction,1)\
println(pressure_int(fcell,interaction))

-16.11425899299892


In [6]:
el=Vector{Float64}([])
lattice_constant = 3.61
cl=1.3:0.02:2.5
en=0
fn=0
fl=Vector{Float64}([])
for lattice_constant in cl
    # 定义铜的FCC晶胞的基矢量
    lattice_vectors = (Matrix([
        lattice_constant 0.0 0.0; #a1
        0.0 lattice_constant 0.0; #a2
        0.0 0.0 lattice_constant] #a3
    ))
    # 创建铜的原子列表
    atoms = [Atom(pos) for pos in atom_positions]

    cell=UnitCell(lattice_vectors,atoms)
    cpcell=copycell(cell,3,3,3)
    fcell=filtercell(cpcell)
    en=cell_energy0(fcell,interaction,ifnormalize=false)
    push!(el,en)
    fn=sum((force_tensor(fcell,interaction)).^2)*fcell.Volume*27
    push!(fl,fn)
end


In [7]:
minlt=cl[argmin(el)]
println(minlt)
lattice_vectors = (Matrix([
    minlt 0.0 0.0; #a1
    0.0 minlt 0.0; #a2
    0.0 0.0 minlt] #a3
))
atoms = [Atom(pos) for pos in atom_positions]
cell=UnitCell(lattice_vectors,atoms)
cpcell=copycell(cell,  5,5,5)
fcell=filtercell(cpcell)
println(cell_energy0(fcell,interaction,ifnormalize=false))
println(force_tensor(fcell,interaction))
println(pressure_int(fcell,interaction))



1.54
-4251.40121010611
[-0.015426277094542525 -5.323943190657902e-16 -1.1496604514712906e-16; 1.0031410602472923e-17 -0.015426277094543018 1.0371870598678063e-16; -7.903535626190786e-18 -6.505217784633956e-18 -0.015426277094543039]
-0.03085255418908571


In [7]:
# 创建一个新的 Figure
fig = Figure()

# 创建一个 Axis，并设置坐标范围
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
vl=cl.^3*(cpcell.copy[1]*cpcell.copy[2]*cpcell.copy[3]);
# lines!(ax,cl,fl)
lines!(ax,cl,fl)
# ylims!(ax, -0.001, 0.001)

fig

In [9]:
# _,fig=BMfit(vl,el,cpcell)
# fig

In [8]:
thermostat = Thermostat(300.0, 10.0, 0.1, 0.0)
barostat=Barostat(10.0,fcell.Volume,10.0,0.0)
println(pressure_int(fcell,interaction))
z=cell2z(fcell,thermostat,barostat);

-0.03085255418908571


In [9]:
zl=[]
tpcell=fcell
z=cell2z(tpcell,thermostat,barostat)
for i in 1:50
    z=RK3_step(z,0.001,tpcell,interaction,thermostat,barostat)
    tpcell=z2cell(z,tpcell)
    println(i)
    push!(zl,z)
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


String: "Warning the same atoms of atom 1 j=2 lead to the nan force,please check their cn"

In [24]:
newcell=z2cell(zl[5],fcell)
println(pressure_int(newcell,interaction))
println(cell_temp(newcell))
fig=visualize_unitcell_atoms(newcell)
fig

-0.30220943404868755
6.093869169885035e24


In [30]:
fig=Figure()
ax = Axis(fig[1, 1], title = "Line Plot", xlabel = "X-Axis", ylabel = "Y-Axis",
          )
pl=[]
for i in 1:30
    newcell=z2cell(zl[i],fcell)
    p=pressure_int(newcell,interaction)
    push!(pl,p)
    

end
ax=lines!(ax,1:30,pl)
fig

In [31]:
pl

30-element Vector{Any}:
   32.916434866204284
   38.8118237293084
   44.56859162355443
   40.95635822687711
   -0.30220943404868755
 -297.6194741064018
  -27.332853576423727
   45.13593251811479
   53.57315927265436
   61.881076283118595
    ⋮
 -379.976359092918
  -59.83063490967683
   19.714008102857868
   35.08665764244202
   58.85087823243603
   91.22304418488007
  125.39316623910855
  149.99593632365404
  153.5295777443471

In [58]:
# Hz(z,fcell,interaction,thermostat,barostat)


In [49]:

# 假设func(i)是一个生成3D图的函数
function func(i)
    newcell=z2cell(zl[i],fcell)
    fig=visualize_unitcell_atoms(newcell)
    return fig
end

# function create_animation()
#     video_file = "outputatom.mp4"
#     n_frames = 100
#     fps = 1
#     # 使用record函数生成动画
#     record(fig, video_file, 1:n_frames; framerate = fps) do i
#         fig=func(i)
#         println("Frame $i")
#     end
# end


# # 调用函数生成视频
# create_animation()

func (generic function with 1 method)

In [50]:
outputfold = "outputfold"
isdir(outputfold) || mkpath(outputfold)
for i in 1:100
    fig=func(i)
    save(joinpath(outputfold, "frame$i.png"), fig)
end

In [11]:
function cell_temp(cell::UnitCell)
    kb=1.38e-23
    Ek=0.0
    for atom in cell.atoms
        p=atom.momentum
        Ek+=sum(p.^2)/(atom.mass)
    end
    return 2*Ek/(3*kb*length(cell.atoms))    
end

cell_temp (generic function with 1 method)

In [31]:
cell_temp(newcell)

7.962750934821664e18